In [0]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import mean_absolute_error as MAE
import gc
import matplotlib
import matplotlib.pyplot as plt
from IPython.display import display, HTML

In [49]:
# Mount personal Google Drive; all data files are in personal Google Drive
# account: My main account @ Gmail
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# Use PDrive function
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [51]:
# List files in root directory
file_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))

title: 23.csv, id: 1JKzKvSw9Ym8iJUzl_JEPuRq-N-PsH1Zx
title: 98, id: 1Wwfy8TyImvrVRktOa3XIknXqKlQVVvQg3PcsbgKD0Ms
title: contacts.csv, id: 1hOS6oFmJDjQQLBY4EnOn9WX1zc-c3xYu
title: https://www.goeugo.com, id: 17JhDtBrBDmyi-QUM35jM5TXoJ6_XdTbN
title: Colab Notebooks, id: 1kXGE13cjX-yz2222w9qTUyQMGPysUgn_
title: 影片集, id: 1cuSzW7boRwQy8nwgxUf_FflAQhmzJhcG
title: 照片集, id: 10c5xL8LrPRdsKAi_8Xaa4K1t29paSGCj
title: 2018卡通簡介30秒版.mp4, id: 1DNw28LhL555PqUZmstvQQvmfTe8aXgZW
title: 2015公司簡介.mp4, id: 1y7yJLy3Bpn919ZIvbmLPOEOBRPgn_9bp
title: 小包團需求_空白表格_2019.docx, id: 1Ft1au82vSv8YO8JY5-olEOh1BDT-dpAk
title: 2019goeugo專業版介紹-同業版_20190612.pptx, id: 170QhaK9XydIvAlQdpJ4FfVmXFnHpASu2
title: 2019歐來歐去循環團.pdf, id: 1mqgf_JBjyXExI5tVKFp55gHGdq7KE_QK
title: 2019歐來歐去桃園說明會-線上報名, id: 17G2uclEvxnd6k3ISEXu0KV_z0pHUMv6yQ1owQYhK4Ks
title: 歐洲報價訊息一手取得_20190612.jpg, id: 1fE4e7mlNp78FtbXVSuZuz8Kb5JrL7Otl


In [53]:
# Check first 5 rows in the airline file
Ydata = pd.read_csv('/content/gdrive/My Drive/23.csv')
Ydata.head()

,001,一陽旅行社股份有限公司,官文清,官文清.1,董事長,03-3336699,03-3336679,桃園市桃園區復興路389號5樓之1,suntour.taoyuan@gmail.com,Unnamed: 9
0,2,八德旅行社有限公司,游景樂,游象新,經理,03-3611445,03-3615153,桃園市八德區東勇北路168巷10弄6號,yupadetravel@hotmail.com,NaN
1,3,力泰旅行社股份有限公司,林秋烈,林秋烈,負責人,03-3258699,03-3556568,桃園市桃園區愛八街5號1樓,a5013156@gmail.com,NaN
2,4,卜鵬國際旅行社有限公司,邱贊因,邱贊因,總經理,03-3588383,03-3170002,桃園市桃園區中正路1003號2樓,cutepei0430@yahoo.com,NaN
3,5,三信旅行社有限公司,張金球,張金球,總經理,03-4258017,03-4222715,桃園市中壢區中光路49號,sanhsintour@gmail.com,NaN
4,6,千菘旅行社有限公司,林彩娥,林彩娥,總經理,03-3370527,03-3385164,桃園市桃園區中正路90-4號5樓,yehlinse@yahoo.com.tw,NaN
